In [1]:
import nengo
import nengo_loihi
import nengo_dl
import numpy as np

import _init_paths
from utils.nengo_dl_utils import get_nengo_dl_model
from utils.base_utils.data_prep_utils import get_exp_dataset
from configs.exp_configs import tf_exp_cfg as tf_cfg, nengo_loihi_cfg as nloihi_cfg
from utils.consts.exp_consts import SEED

In [2]:
inpt_shape, num_clss = (32, 32, 3), 10

ndl_model, ngo_probes_lst = get_nengo_dl_model(inpt_shape, tf_cfg, nloihi_cfg, mode="test", num_clss=num_clss)
ndl_model.model.summary()

RG: Input shape: (32, 32, 3)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 4)         12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 8)         288       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         576       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 16)        1152      
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (No

# Load dataset

In [3]:
_, _, test_x, test_y = get_exp_dataset(
    nloihi_cfg["dataset"], channels_first=tf_cfg["is_channels_first"],start_idx=75, end_idx=150)
print("Before Flattening: ", test_x.shape, test_y.shape)
test_x = test_x.reshape((test_x.shape[0], 1, -1))
print("After Flattening: ", test_x.shape)

pres_steps = nloihi_cfg["test_mode"]["n_steps"]
pres_time = nloihi_cfg["test_mode"]["n_steps"] * 0.001
print("Presentation Time: ", pres_time)

Before Flattening:  (75, 32, 32, 3) (75, 10)
After Flattening:  (75, 1, 3072)
Presentation Time:  0.1


# Load the NDL trained weights

In [4]:
nengo_input, nengo_output = ngo_probes_lst[0], ngo_probes_lst[-1]
with nengo_dl.Simulator(ndl_model.net, seed=SEED) as ndl_sim:
  ndl_sim.load_params(nloihi_cfg["trained_model_params"]+ "/ndl_trained_params")
  ndl_sim.freeze_params(ndl_model.net)
  
with ndl_model.net:
  nengo_input.output = nengo.processes.PresentInput(test_x, presentation_time=pres_time)
  nengo_loihi.add_params(ndl_model.net)
  ndl_model.net.config[ndl_model.layers[ndl_model.model.layers[1]].ensemble].on_chip = False

Build finished in 0:00:00                                                      
|#             Optimizing graph: operator simplificaton               | 0:00:00

/homes/rgaurav/nxsdk_1_nengo_loihi/lib/python3.8/site-packages/nengo_dl/simulator.py:460: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(


Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


# Set Block Shape of Conv Layers

In [5]:
bls_dict, i = nloihi_cfg["layer_blockshapes"], 0
with ndl_model.net:
  # Exclude the Conv layer at index 1 since it runs off-chip to create spikes.
  for layer in ndl_model.model.layers[2:-1]:
    if layer.name.startswith("conv"):
      conv_shape = tuple(layer.output.shape[1:])
      if np.prod(conv_shape) <= 1024:
        continue
      ndl_model.net.config[
      ndl_model.layers[layer].ensemble].block_shape = (
      nengo_loihi.BlockShape(bls_dict["conv2d_%s" % i], conv_shape))
      i+=1

# Make Predictions

In [6]:
with nengo_loihi.Simulator(ndl_model.net, seed=SEED, target="loihi", precompute=False) as loihi_sim:
  loihi_sim.run(nloihi_cfg["test_mode"]["n_test"] * pres_time)

/homes/rgaurav/nengo-loihi/nengo_loihi/simulator.py:147: UserWarning: Model is precomputable. Setting precompute=False may slow execution.
  warnings.warn(


INFO:DRV:  SLURM is being run in background


INFO:DRV:SLURM is being run in background


INFO:DRV:  Connecting to 10.212.98.178:46279


INFO:DRV:Connecting to 10.212.98.178:46279


...

INFO:HST:[Host] Wrote superhost shutdown signal: 8192 bytes


INFO:HST:  [Host] Closing server socket


INFO:HST:[Host] Closing server socket


INFO:HST:  chip=0 cpu=0 halted, status=0x0


INFO:HST:chip=0 cpu=0 halted, status=0x0


In [7]:
output = loihi_sim.data[nengo_output]
np.save("all_conv_loihi_pred_75_150_test_images", output)

In [8]:
loihi_pred_clss = np.argmax(output[pres_steps-1 :: pres_steps], axis=-1)
acc = 100 * np.mean(loihi_pred_clss == np.argmax(test_y[:nloihi_cfg["test_mode"]["n_test"]], axis=-1))
print(acc)

57.333333333333336


# Calculate predictions of 0 to 100 test images

In [4]:
pred_0_75 = np.load("all_conv_loihi_pred_0_75_test_images.npy")
pred_75_150 = np.load("all_conv_loihi_pred_75_150_test_images.npy")
pred_0_75 = np.argmax(pred_0_75[pres_steps-1 :: pres_steps], axis=-1)
pred_75_150 = np.argmax(pred_75_150[pres_steps-1 :: pres_steps], axis=-1)

In [5]:
lst = []
lst.extend(pred_0_75)
lst.extend(pred_75_150)
_, _, _, test_y = get_exp_dataset(
    nloihi_cfg["dataset"], channels_first=tf_cfg["is_channels_first"],start_idx=0, end_idx=100)
print(100 * np.mean(lst[:100] == np.argmax(test_y[:], axis=-1)))

59.0
